In [ ]:
# Document에 나온 버전 
# Reactor 충돌이 일어나 매번 커널을 재시작하고 Clear all Output 해야 함.
from scrapy.crawler import CrawlerProcess
from danawa_crawler.spiders.danawa_spider import DanawaSpider
from scrapy.utils.project import get_project_settings


def start_crawling(search_keyword="제로 음료", pages=4):
    settings = get_project_settings()
    process = CrawlerProcess(settings=settings)
    process.crawl(DanawaSpider, search_keyword=search_keyword, pages=pages)
    process.start()
    return "크롤링 시작됨"

start_crawling("제로 콜라", 4)

In [ ]:
# 파일 만들어서 별로 프로세스에서 실행하는 버전
import os
import sys
import subprocess

def run_crawler_in_separate_process(search_keyword, pages):
    """
    별도의 Python 프로세스에서 크롤러를 실행하는 함수
    """
    # 크롤링 스크립트 생성
    script_content = f'''
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath(__file__)))

from scrapy.crawler import CrawlerProcess
from danawa_crawler.spiders.danawa_spider import DanawaSpider
from scrapy.utils.project import get_project_settings

def main():
    settings = get_project_settings()
    process = CrawlerProcess(settings=settings)
    process.crawl(DanawaSpider, search_keyword="{search_keyword}", pages={pages})
    process.start()

if __name__ == "__main__":
    main()
'''
    
    # 임시 스크립트 파일 생성
    script_path = "temp_crawler_script.py"
    with open(script_path, 'w', encoding='utf-8') as f:
        f.write(script_content)
    
    try:
        # 별도 프로세스에서 스크립트 실행
        result = subprocess.run([sys.executable, script_path], 
                              capture_output=True, text=True, timeout=300)
        
        if result.returncode == 0:
            print("크롤링이 성공적으로 완료되었습니다!")
            return "크롤링 완료"
        else:
            print(f"크롤링 중 오류: {result.stderr}")
            return f"오류: {result.stderr}"
            
    except subprocess.TimeoutExpired:
        return "크롤링 시간 초과"
    except Exception as e:
        return f"오류: {str(e)}"
    finally:
        # 임시 파일 삭제
        if os.path.exists(script_path):
            os.remove(script_path)

# 크롤링 실행
result = run_crawler_in_separate_process("제로 슈거", 4)
print(result)

In [ ]:
# commend 에서 실행하는 코드

import subprocess
import os

def run_scrapy_crawling(search_keyword="제로 콜라", pages=4):
    try:
        command = f"scrapy crawl danawa -a search_keyword='{search_keyword}' -a pages={pages}"
        
        result = subprocess.run(
            command,
            shell=True,
            capture_output=True,
            text=True,
            timeout=300
        )
        
        if result.returncode == 0:
            return {"status": "success", "message": "크롤링 완료", "output": result.stdout}
        else:
            return {"status": "error", "message": result.stderr}
            
    except Exception as e:
        return {"status": "error", "message": str(e)}

# 사용 예시
result = run_scrapy_crawling("그릭 요거트", 4)
print(result)

In [ ]:
os.getcwd()